In [1]:
import numpy as np
import pandas as pd
import cv2
import json
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
## image processing functions to locate game board and label it ##

def findBoard(image, threshold):
    """
    Finds location of game baord given input image and crops to the game board.

    Args:
        image: BGR image (as read by cv2.imread)
        threshhold: low threshold value for Canny edge detection
        
    Returns:
        board_region: BGR image cropped to the game board
    """
    
    # convert to gray
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # detect edges
    edges = cv2.Canny(gray, threshold, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # crop to game board
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    board_region = image[y:y+h, x:x+w] 

    return board_region


def extractHexVectors(board, xy_to_cubic, color_ranges):
    """
    Extracts color counts for each hex on the game board and outputs them as a vector, using cubic coordinates.

    Args:
        board: BGR image cropped to the board (OpenCV format)
        xy_to_cubic: List of dicts with 'pixel': [x, y], 'cubic': [q, r, s]
        color_ranges: Dict of token_type -> (lower_bgr, upper_bgr)

    Returns:
        List of dicts: [{'coord': [q, r, s], 'vector': {'red': ..., 'blue': ..., ...}}, ...]
    """

    # Convert to RGB
    board = cv2.cvtColor(board, cv2.COLOR_BGR2RGB)

    # Convert list of dicts to a proper mapping for lookup
    xy_to_cubic_dict = {
        tuple(entry["pixel"]): tuple(entry["cubic_coord"])
        for entry in xy_to_cubic
    }

    vectors = []

    for (hex_x, hex_y), cubic_coord in xy_to_cubic_dict.items():
        roi = board[hex_y - 25: hex_y + 25, hex_x - 10: hex_x + 10]  # Region of interest
        vector = {}

        for token_type, (lower, upper) in color_ranges.items():
            lower_np = np.array(lower, dtype=np.uint8)
            upper_np = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(roi, lower_np, upper_np)
            pixel_count = cv2.countNonZero(mask)

            # Special adjustment for spirit token (similar color to board)
            if token_type == "spirit":
                board_mask = cv2.inRange(roi, np.array(color_ranges["board"][0]), np.array(color_ranges["board"][1]))
                pixel_count = max(0, pixel_count - cv2.countNonZero(board_mask))

            vector[token_type] = int(pixel_count)

        vectors.append({
            "coord": list(cubic_coord),  # Output in [q, r, s]
            "vector": vector
        })

    return vectors



def classify_vector(input_vector, token_avgs):
    """
    Given a pixel count vector, return the top matching label from a vector database of average values.
    
    Args:
        input_vector (dict): e.g., {'red': 0, 'blue': 172, ..., 'animal': 133}
        token_vectors (pd.DataFrame) - database of tokens vectors, averaged from labeled data

    Returns:
        list of tuples: [(label, similarity_score), ...]
    """
    
    # ensure vectors are in the same order
    token_order = token_avgs.index.tolist()
    input_array = np.array([[input_vector.get(token, 0) for token in token_order]])  # shape (1, n_tokens)

    # transpose for cosine similarity: now shape (n_labels, n_tokens)
    label_vectors = token_avgs.T.values
    label_names = token_avgs.columns.tolist()

    # cosine similarity
    similarities = cosine_similarity(input_array, label_vectors)[0]
    best_index = similarities.argmax()

    return label_names[best_index]

def convert_to_board_dict(label_list):
    """
    Convert list of {'coord': [...], 'label': 'token1, token2, ...'} 
    to { (q, r, s): [token1, token2, ...], ... }
    """
    board_dict = {}
    for entry in label_list:
        coord = tuple(entry["coord"])
        label = entry["label"]
        if label.lower() == "none":
            board_dict[coord] = []
        else:
            tokens = [token.strip() for token in label.split(",")]
            board_dict[coord] = tokens
    return board_dict


In [3]:
## functions to match the labeled board with spirit and animal patterns ##

def rotate_cubic(dq, dr, ds, times=1):
    for _ in range(times % 6):
        dq, dr, ds = -dr, -ds, -dq
    return dq, dr, ds

def rotate_pattern(pattern, times):
    """Rotate spirit/animal pattern"""
    rotated = []
    for entry in pattern:
        dq, dr, ds = entry["dq"], entry["dr"], entry["ds"]
        rotated_entry = dict(entry)  # shallow copy
        rotated_entry["dq"], rotated_entry["dr"], rotated_entry["ds"] = rotate_cubic(dq, dr, ds, times)
        rotated.append(rotated_entry)
    return rotated

def clean_stack(stack):
    """Remove animal/spirit cubes from the top of the stack (if present)."""
    return [item for item in stack if item not in ("animal", "spirit")]

def match_pattern2(board_dict, anchor_coord, pattern):
    """
    Check if a rotated pattern matches the board at a given anchor coordinate.

    Args:
        board_dict: dict with keys as (q, r, s) and values as stack lists (bottom to top)
        anchor_coord: (q, r, s) tuple
        pattern: List of dicts with dq, dr, ds and terrain/stack_condition/cube

    Returns:
        True if pattern matches, False otherwise
    """
    for entry in pattern:
        print('Testing pattern entry:', entry)
        dq, dr, ds = entry["dq"], entry["dr"], entry["ds"]
        q, r, s = anchor_coord[0] + dq, anchor_coord[1] + dr, anchor_coord[2] + ds
        coord = (q, r, s)
        print(f'Coordinate tested: {coord}')
        
        stack = board_dict.get(coord, [])
        print(f'Stack before cleaning: {stack}')

        cleaned_stack = clean_stack(stack)
        print(f'Stack after cleaning: {cleaned_stack}')

        # Check cube requirement
        if entry.get("cube", False):
            if not stack or stack[0] not in ["animal", "spirit"]:
                print(f"❌ Cube not found where required at {coord}")
                return False

        # Check simple terrain match
        if "terrain" in entry:
            terrain_matches = any(tok == entry["terrain"] for tok in cleaned_stack)
            if not terrain_matches:
                print(f"❌ Terrain mismatch at {coord}. Expected {entry['terrain']}, got {cleaned_stack}")
                return False

        # Check stack condition match
        if "stack_condition" in entry:
            sc = entry["stack_condition"]
            print('stack_condition:')
            print(sc)
            print('cleaned_stack:')
            print(cleaned_stack)
            if len(cleaned_stack) < sc.get("height", 0):
                print(f"❌ Stack too short at {coord}. Needed height {sc.get('height')}, got {len(cleaned_stack)}")
                return False

            # Check top
            top_matches = cleaned_stack[0] == sc["top"]
            print(f'Top matches: {top_matches}')

            # Optional middle
            middle_matches = True
            if "middle" in sc and len(cleaned_stack) > 1:
                middle_matches = cleaned_stack[1] == sc["middle"]
                print(f'Middle matches or NA: {middle_matches}')

            # Check bottom
            bottom_matches = cleaned_stack[-1] in sc["bottom_options"]
            print(f'Bottom matches: {bottom_matches}')

            if not (top_matches and middle_matches and bottom_matches):
                print(f"❌ Stack structure mismatch at {coord}. Stack: {cleaned_stack}")
                return False

    return True  # All checks passed
def match_pattern_any_rotation(board_dict, pattern):
    """
    Try all 6 rotations of the pattern at every hex in the board_dict.

    Returns:
        Tuple (True, matched_anchor_coord) if a match is found
        Else, (False, None)
    """
    for anchor_coord in board_dict.keys():
        print(f'Testing hex at position: {anchor_coord}')
        for rotation in range(6):
            rotated = rotate_pattern(pattern, rotation)

            print(f"\n🔄 Trying rotation {rotation} at anchor {anchor_coord}")
            print("Rotated Pattern:")
            for p in rotated:
                print(p)
            
            if match_pattern2(board_dict, anchor_coord, rotated):
                return True, anchor_coord
    return False, None


In [4]:
# load image and other necessary data
image = cv2.imread('C:/Users/lacto/Documents/GitHub/HarmoniesRender/training_boards/screenshots/Board4.png')

# get reference coordinates for hexagon centers
f = open('hex_positions_cubic.json')
hex_centers = json.load(f)

# get reference patterns
f = open('spirit_patterns_cubic.json')
spirit_patterns = json.load(f)

# load token color ranges
f = open('token_color_ranges.json')
color_ranges = json.load(f)

# load token vectors
token_avgs = pd.read_csv('average_token_vectors.csv', index_col=0)

In [5]:
## MAIN ##

# crop image to board state
board = findBoard(image, threshold = 50)

# count number of pixels in each token's color range for each board hex
hex_vectors = extractHexVectors(board, hex_centers, color_ranges)

# use pixel counts to detect tokens, cubes, and the stack order
labels = []
for hex in hex_vectors: # for each board hex
    input_vector = hex["vector"] # get vector of color counts
    label = classify_vector(input_vector, token_avgs) # use cosine similarity to classify vector
    labels.append({
        "coord": hex["coord"],
        "label": label
    }) 

board_dict = convert_to_board_dict(labels)

# detect spirit placement
spirit_coord = None
matched_spirit = None

for pattern in spirit_patterns:
    print(f'Testing pattern: {pattern}')
    matched, coord = match_pattern_any_rotation(board_dict, pattern["pattern"])
    if matched:
        spirit_coord = coord
        matched_spirit = pattern["animal"]
        print(f"✅ Found match: {matched_spirit} at {spirit_coord}")
        break

if spirit_coord is None:
    print("❌ No matching spirit pattern found.")
    
## GEN AI ##
    
# infer terrain and habitat features from
# - Spirt Card
# - Animal Cards
# - Tokens (Green, Yellow, Blue, Gray, Red, Brown)
# - Token density

# infer arrangement of features 

# combine inferences with stylistic preferences

# create image

# save image

Testing pattern: {'animal': 'domestic cat (spirit)', 'pattern': [{'terrain': 'green', 'dq': 0, 'dr': 0, 'ds': 0}, {'stack_condition': {'height': 2, 'top': 'red', 'bottom_options': ['gray', 'brown', 'red']}, 'dq': -1, 'dr': 0, 'ds': 1}, {'stack_condition': {'height': 2, 'top': 'red', 'bottom_options': ['gray', 'brown', 'red']}, 'cube': True, 'dq': 1, 'dr': 0, 'ds': -1}]}
Testing hex at position: (-2, -1, 3)

🔄 Trying rotation 0 at anchor (-2, -1, 3)
Rotated Pattern:
{'terrain': 'green', 'dq': 0, 'dr': 0, 'ds': 0}
{'stack_condition': {'height': 2, 'top': 'red', 'bottom_options': ['gray', 'brown', 'red']}, 'dq': -1, 'dr': 0, 'ds': 1}
{'stack_condition': {'height': 2, 'top': 'red', 'bottom_options': ['gray', 'brown', 'red']}, 'cube': True, 'dq': 1, 'dr': 0, 'ds': -1}
Testing pattern entry: {'terrain': 'green', 'dq': 0, 'dr': 0, 'ds': 0}
Coordinate tested: (-2, -1, 3)
Stack before cleaning: ['animal', 'green', 'brown']
Stack after cleaning: ['green', 'brown']
Testing pattern entry: {'stack_

In [8]:
print(spirit_coord)
print(matched_spirit)

(0, -1, 1)
dragonfly (spirit)


In [7]:
# something wrong with the rotations in the new match function